## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
#pip install config

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps



# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_data/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Country,Date
0,0,Karratha,-20.7377,116.8463,85.33,44,42,5.66,light rain,AU,2022-04-17 15:47:47
1,1,Hermanus,-34.4187,19.2345,67.80,59,9,15.57,clear sky,ZA,2022-04-17 15:47:47
2,2,Bredasdorp,-34.5322,20.0403,63.43,72,0,9.44,clear sky,ZA,2022-04-17 15:47:47
3,3,Hilo,19.7297,-155.0900,73.27,95,100,3.44,light rain,US,2022-04-17 15:47:48
4,4,Abonnema,4.7084,6.7931,88.48,60,93,6.78,overcast clouds,NG,2022-04-17 15:47:48


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Country,Date
0,0,Karratha,-20.7377,116.8463,85.33,44,42,5.66,light rain,AU,2022-04-17 15:47:47
1,1,Hermanus,-34.4187,19.2345,67.80,59,9,15.57,clear sky,ZA,2022-04-17 15:47:47
2,2,Bredasdorp,-34.5322,20.0403,63.43,72,0,9.44,clear sky,ZA,2022-04-17 15:47:47
3,3,Hilo,19.7297,-155.0900,73.27,95,100,3.44,light rain,US,2022-04-17 15:47:48
4,4,Abonnema,4.7084,6.7931,88.48,60,93,6.78,overcast clouds,NG,2022-04-17 15:47:48
6,6,Cayenne,4.9333,-52.3333,84.24,14,100,11.50,overcast clouds,GF,2022-04-17 15:47:49
7,7,Matai,-8.3000,31.5167,78.24,93,17,4.23,light rain,TZ,2022-04-17 15:47:49
8,8,Lynchburg,37.4138,-79.1423,63.88,32,0,8.05,clear sky,US,2022-04-17 15:47:50
11,11,Barkly West,-28.5354,24.5215,63.68,65,57,12.59,light rain,ZA,2022-04-17 15:47:51
12,12,Rikitea,-23.1203,-134.9692,78.15,65,3,12.37,clear sky,PF,2022-04-17 15:39:40


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        375
City           375
Lat            375
Lng            375
Max Temp       375
Humidity       375
Cloudiness     375
Wind Speed     375
Description    375
Country        371
Date           375
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
len(preferred_cities_df)
#### NO EMPTY ROWS 


375

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Karratha,AU,85.33,light rain,-20.7377,116.8463,
1,Hermanus,ZA,67.80,clear sky,-34.4187,19.2345,
2,Bredasdorp,ZA,63.43,clear sky,-34.5322,20.0403,
3,Hilo,US,73.27,light rain,19.7297,-155.0900,
4,Abonnema,NG,88.48,overcast clouds,4.7084,6.7931,
6,Cayenne,GF,84.24,overcast clouds,4.9333,-52.3333,
7,Matai,TZ,78.24,light rain,-8.3000,31.5167,
8,Lynchburg,US,63.88,clear sky,37.4138,-79.1423,
11,Barkly West,ZA,63.68,light rain,-28.5354,24.5215,
12,Rikitea,PF,78.15,clear sky,-23.1203,-134.9692,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
   
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError) as e:
        print(e)
        print("Hotel not found... skipping.")
        
print("Processing Complete")

list index out of range
Hotel not found... skipping.
list index out of range
Hotel not found... skipping.
list index out of range
Hotel not found... skipping.
list index out of range
Hotel not found... skipping.
list index out of range
Hotel not found... skipping.
list index out of range
Hotel not found... skipping.
list index out of range
Hotel not found... skipping.
list index out of range
Hotel not found... skipping.
list index out of range
Hotel not found... skipping.
list index out of range
Hotel not found... skipping.
list index out of range
Hotel not found... skipping.
list index out of range
Hotel not found... skipping.
list index out of range
Hotel not found... skipping.
list index out of range
Hotel not found... skipping.
list index out of range
Hotel not found... skipping.
list index out of range
Hotel not found... skipping.
list index out of range
Hotel not found... skipping.
list index out of range
Hotel not found... skipping.
list index out of range
Hotel not found... ski

In [16]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df[hotel_df["Hotel Name"].notna()]
hotel_df = hotel_df[hotel_df["Country"].notna()]
hotel_df.count()

City           371
Country        371
Max Temp       371
Description    371
Lat            371
Lng            371
Hotel Name     371
dtype: int64

In [17]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]


In [19]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))